In [11]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os

print(tf.__version__)

2.3.0


In [12]:
# checkpoint function
# 비정상적으로 종료되거나, 재시작 시사용한다
# checkpoint에서 변수를 다시 로드하여 학습

def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

In [13]:
# Data 로드
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels
# pre-processing function (normalization)
# normalization : data의 값 범위를 0 ~ 1 로 만들어 준다
def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

In [14]:
# cost function
def loss_fn(model, images, labels):
    logits = model(images, training = True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

# 정확도 
def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    #argmax : 제일 높은 값의 위치
    #예측값과 실제값이 같은 지 비교해 본다
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    #계산을 위해 예측결과를 float로 casting 해준다
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

In [15]:
# 1개의 행으로 바꿔주는 함수
# shape을 펼쳐주는 역할
def flatten() :
    return tf.keras.layers.Flatten()

# dense layer = fully connected layer
# units :  ouput으로 나가는 채널을 몇 개로 설정할 것인가
# use_bias : bias를 쓸 것인가
# kernel_initializer : weight initializer 라고 생각하면 된다
def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def sigmoid():
    return tf.keras.layers.Activation(tf.keras.activations.sigmoid)

In [16]:
# create model (class version)
class create_model_class(tf.keras.Model):
    # label_dim은 output의 갯수 이다
    # mnist에서는 10
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.RandomNormal()
        self.model = tf.keras.Sequential()
        self.model = add(flatten())
        
        for i in range(2):
            self.model.add(dense(256, weight_init))
            self.model.add(sigmoid())
        self.model.add(dense(label_dim, weight_init))
    
    def call(self, x, training = None, mask = None):
        x = self.model(x)
        return x

In [17]:
# create model (function version)

def create_model_function(label_dim):
    weight_init = tf.keras.initializers.RandomNormal()
    model = tf.keras.Sequential()
    model.add(flatten())
    
    for i in range(2) :
        model.add(dense(256, weight_init))
        model.add(sigmoid())
    model.add(dense(label_dim, weight_init))
    
    return model

In [18]:
# dataset
train_x, train_y, test_x, test_y = load_mnist()

# parameters
learning_rate = 0.001
batch_size = 128

training_epochs = 1
# batch_size
training_iterations = len(train_x)

# output 갯수
label_dim = 10

train_flag = True

# 한번에 많은 데이터를 학습시키면 무리가 가기 때문에
# batch size 만큼씩만 학습시킨다
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

In [19]:
# model
# 함수 이용
network = create_model_function(label_dim)

# training
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)

# writer
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_softmax'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

In [20]:
# restore checkpoint
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train
    with summary_writer.as_default(): #for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))
                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)
                for test_input, test_label in test_dataset:
                    test_accuracy = accuracy_fn(network, test_input, test_label)
                tf.summary.scalar(name = 'train_loss', data=train_loss,step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data = test_accuracy, step=counter)
                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1
                checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
    
    # test
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:
        test_accuracy = accuracy_fn(network, test_input, test_label)
    print("test_Accuracy: %.4f" % (test_accarucy))

 [*] Reading checkpoints...
 [*] Success to read nn_softmax-468-468
 [*] Load SUCCESS
Epoch: [ 0] [    0/60000] time: 1.8345, train_loss: 0.22495024, train_accuracy: 0.9453, test_Accuracy: 0.9245
Epoch: [ 0] [    1/60000] time: 2.1796, train_loss: 0.26913548, train_accuracy: 0.9141, test_Accuracy: 0.9206
Epoch: [ 0] [    2/60000] time: 2.5162, train_loss: 0.19846551, train_accuracy: 0.9453, test_Accuracy: 0.9200
Epoch: [ 0] [    3/60000] time: 2.9201, train_loss: 0.21897045, train_accuracy: 0.9375, test_Accuracy: 0.9220
Epoch: [ 0] [    4/60000] time: 3.3162, train_loss: 0.20200014, train_accuracy: 0.9531, test_Accuracy: 0.9239
Epoch: [ 0] [    5/60000] time: 3.6224, train_loss: 0.21765058, train_accuracy: 0.9453, test_Accuracy: 0.9242
Epoch: [ 0] [    6/60000] time: 3.9356, train_loss: 0.37790862, train_accuracy: 0.8828, test_Accuracy: 0.9245
Epoch: [ 0] [    7/60000] time: 4.3385, train_loss: 0.28344926, train_accuracy: 0.9141, test_Accuracy: 0.9234
Epoch: [ 0] [    8/60000] time: 4.

Epoch: [ 0] [   74/60000] time: 30.6440, train_loss: 0.24991000, train_accuracy: 0.9297, test_Accuracy: 0.9265
Epoch: [ 0] [   75/60000] time: 30.9920, train_loss: 0.24354878, train_accuracy: 0.9453, test_Accuracy: 0.9285
Epoch: [ 0] [   76/60000] time: 31.4924, train_loss: 0.15197279, train_accuracy: 0.9688, test_Accuracy: 0.9281
Epoch: [ 0] [   77/60000] time: 31.9372, train_loss: 0.23722038, train_accuracy: 0.9141, test_Accuracy: 0.9289
Epoch: [ 0] [   78/60000] time: 32.2857, train_loss: 0.25169003, train_accuracy: 0.9375, test_Accuracy: 0.9278
Epoch: [ 0] [   79/60000] time: 32.6098, train_loss: 0.18101591, train_accuracy: 0.9609, test_Accuracy: 0.9284
Epoch: [ 0] [   80/60000] time: 32.9599, train_loss: 0.34018975, train_accuracy: 0.9375, test_Accuracy: 0.9285
Epoch: [ 0] [   81/60000] time: 33.2980, train_loss: 0.19827394, train_accuracy: 0.9219, test_Accuracy: 0.9288
Epoch: [ 0] [   82/60000] time: 33.7523, train_loss: 0.27896196, train_accuracy: 0.9219, test_Accuracy: 0.9293
E

Epoch: [ 0] [  148/60000] time: 58.6464, train_loss: 0.21818182, train_accuracy: 0.9375, test_Accuracy: 0.9347
Epoch: [ 0] [  149/60000] time: 58.9675, train_loss: 0.17847945, train_accuracy: 0.9531, test_Accuracy: 0.9343
Epoch: [ 0] [  150/60000] time: 59.3036, train_loss: 0.33501318, train_accuracy: 0.9219, test_Accuracy: 0.9344
Epoch: [ 0] [  151/60000] time: 59.6786, train_loss: 0.16515136, train_accuracy: 0.9609, test_Accuracy: 0.9345
Epoch: [ 0] [  152/60000] time: 60.0042, train_loss: 0.17716016, train_accuracy: 0.9609, test_Accuracy: 0.9353
Epoch: [ 0] [  153/60000] time: 60.3652, train_loss: 0.18753272, train_accuracy: 0.9453, test_Accuracy: 0.9357
Epoch: [ 0] [  154/60000] time: 60.8437, train_loss: 0.19468330, train_accuracy: 0.9219, test_Accuracy: 0.9350
Epoch: [ 0] [  155/60000] time: 61.5907, train_loss: 0.22717658, train_accuracy: 0.9453, test_Accuracy: 0.9363
Epoch: [ 0] [  156/60000] time: 62.0903, train_loss: 0.14062476, train_accuracy: 0.9688, test_Accuracy: 0.9370
E

Epoch: [ 0] [  222/60000] time: 91.1710, train_loss: 0.18919474, train_accuracy: 0.9453, test_Accuracy: 0.9347
Epoch: [ 0] [  223/60000] time: 91.4981, train_loss: 0.19424772, train_accuracy: 0.9297, test_Accuracy: 0.9353
Epoch: [ 0] [  224/60000] time: 91.9854, train_loss: 0.16109496, train_accuracy: 0.9531, test_Accuracy: 0.9370
Epoch: [ 0] [  225/60000] time: 92.2961, train_loss: 0.34495437, train_accuracy: 0.8984, test_Accuracy: 0.9371
Epoch: [ 0] [  226/60000] time: 92.6152, train_loss: 0.21566805, train_accuracy: 0.9609, test_Accuracy: 0.9381
Epoch: [ 0] [  227/60000] time: 92.9244, train_loss: 0.27961785, train_accuracy: 0.9141, test_Accuracy: 0.9383
Epoch: [ 0] [  228/60000] time: 93.2486, train_loss: 0.22548094, train_accuracy: 0.9375, test_Accuracy: 0.9380
Epoch: [ 0] [  229/60000] time: 93.5877, train_loss: 0.25908619, train_accuracy: 0.9297, test_Accuracy: 0.9381
Epoch: [ 0] [  230/60000] time: 93.8958, train_loss: 0.20182648, train_accuracy: 0.9375, test_Accuracy: 0.9382
E

Epoch: [ 0] [  296/60000] time: 117.7092, train_loss: 0.18770859, train_accuracy: 0.9531, test_Accuracy: 0.9394
Epoch: [ 0] [  297/60000] time: 118.0293, train_loss: 0.19924770, train_accuracy: 0.9375, test_Accuracy: 0.9407
Epoch: [ 0] [  298/60000] time: 118.3178, train_loss: 0.16888916, train_accuracy: 0.9531, test_Accuracy: 0.9398
Epoch: [ 0] [  299/60000] time: 118.6200, train_loss: 0.25197110, train_accuracy: 0.9297, test_Accuracy: 0.9403
Epoch: [ 0] [  300/60000] time: 118.9256, train_loss: 0.17761418, train_accuracy: 0.9375, test_Accuracy: 0.9399
Epoch: [ 0] [  301/60000] time: 119.2278, train_loss: 0.14862411, train_accuracy: 0.9531, test_Accuracy: 0.9388
Epoch: [ 0] [  302/60000] time: 119.5320, train_loss: 0.29908827, train_accuracy: 0.9141, test_Accuracy: 0.9380
Epoch: [ 0] [  303/60000] time: 119.8601, train_loss: 0.22565123, train_accuracy: 0.9531, test_Accuracy: 0.9383
Epoch: [ 0] [  304/60000] time: 120.2301, train_loss: 0.11924911, train_accuracy: 0.9766, test_Accuracy:

Epoch: [ 0] [  370/60000] time: 143.4178, train_loss: 0.20187545, train_accuracy: 0.9531, test_Accuracy: 0.9416
Epoch: [ 0] [  371/60000] time: 143.7265, train_loss: 0.21602008, train_accuracy: 0.9453, test_Accuracy: 0.9420
Epoch: [ 0] [  372/60000] time: 144.0865, train_loss: 0.24659744, train_accuracy: 0.9297, test_Accuracy: 0.9432
Epoch: [ 0] [  373/60000] time: 144.4216, train_loss: 0.27073634, train_accuracy: 0.9141, test_Accuracy: 0.9423
Epoch: [ 0] [  374/60000] time: 144.7158, train_loss: 0.27563995, train_accuracy: 0.9375, test_Accuracy: 0.9427
Epoch: [ 0] [  375/60000] time: 145.0076, train_loss: 0.11005768, train_accuracy: 0.9688, test_Accuracy: 0.9426
Epoch: [ 0] [  376/60000] time: 145.3187, train_loss: 0.14284974, train_accuracy: 0.9688, test_Accuracy: 0.9426
Epoch: [ 0] [  377/60000] time: 145.6149, train_loss: 0.13917801, train_accuracy: 0.9688, test_Accuracy: 0.9427
Epoch: [ 0] [  378/60000] time: 145.9211, train_loss: 0.20865837, train_accuracy: 0.9609, test_Accuracy:

Epoch: [ 0] [  444/60000] time: 171.7824, train_loss: 0.10720687, train_accuracy: 0.9844, test_Accuracy: 0.9461
Epoch: [ 0] [  445/60000] time: 172.0895, train_loss: 0.24741970, train_accuracy: 0.9062, test_Accuracy: 0.9453
Epoch: [ 0] [  446/60000] time: 172.3797, train_loss: 0.26425195, train_accuracy: 0.9141, test_Accuracy: 0.9446
Epoch: [ 0] [  447/60000] time: 172.7259, train_loss: 0.10532258, train_accuracy: 0.9609, test_Accuracy: 0.9443
Epoch: [ 0] [  448/60000] time: 173.0642, train_loss: 0.21369493, train_accuracy: 0.8906, test_Accuracy: 0.9443
Epoch: [ 0] [  449/60000] time: 173.4307, train_loss: 0.09865928, train_accuracy: 0.9688, test_Accuracy: 0.9434
Epoch: [ 0] [  450/60000] time: 173.8925, train_loss: 0.12372409, train_accuracy: 0.9688, test_Accuracy: 0.9436
Epoch: [ 0] [  451/60000] time: 174.3513, train_loss: 0.21986307, train_accuracy: 0.9375, test_Accuracy: 0.9438
Epoch: [ 0] [  452/60000] time: 174.8101, train_loss: 0.10949792, train_accuracy: 0.9688, test_Accuracy: